# Helper Functions

In [1]:
from os import listdir
from os.path import isfile, join
import pandas as pd

## Create csv-files for PyTorch Dataclass

In [6]:
def create_info_csv(path, split=True):
    fnames = [f for f in listdir(path) if isfile(join(path, f))]
    fnames = [f.split('.',1)[0] for f in fnames]
    fnames_frame = pd.DataFrame(fnames, columns=['fnames'])

    if split:
        new = fnames_frame['fnames'].str.split("_", n = 2, expand = True)


        # making separate first name column from new data frame
        fnames_frame["wid"]= new[0]

        # making separate last name column from new data frame
        fnames_frame["pid"]= new[1]

        # making separate last name column from new data frame
        fnames_frame["fid"]= new[2]

        # Dropping old Name columns
        fnames_frame.drop(columns =['fnames'], inplace = True)

    # save as csf
    info_path = join(path, 'info.csv')
    fnames_frame.to_csv(info_path)

In [5]:
create_info_csv('/Users/beantown/PycharmProjects/jigsaw-puzzle-solver/data/hisfrag20/raw/hisfrag20')
create_info_csv('/Users/beantown/PycharmProjects/jigsaw-puzzle-solver/data/hisfrag20/raw/hisfrag20_test')

In [ ]:

# Identifying prominent colors from image

# Import Libraries
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import cv2
from collections import Counter
from skimage.color import rgb2lab, deltaE_cie76
import os

# (R,G,B) to hexadecimal converter function
def RGB2HEX(color):
    return "#{:02x}{:02x}{:02x}".format(int(color[0]), int(color[1]), int(color[2]))

# Image Path utility function
def get_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

# Function to get prominent colors from image
def get_colors(image, number_of_colors, show_chart):

    # Modifies image to usable format
    modified_image = cv2.resize(image, (600, 400), interpolation = cv2.INTER_AREA)
    modified_image = modified_image.reshape(modified_image.shape[0]*modified_image.shape[1], 3)

    # Clusters color regions
    clf = KMeans(n_clusters = number_of_colors)
    labels = clf.fit_predict(modified_image)

    counts = Counter(labels)
    center_colors = clf.cluster_centers_

    # We get ordered colors by iterating through the keys
    ordered_colors = [center_colors[i] for i in counts.keys()]
    hex_colors = [RGB2HEX(ordered_colors[i]) for i in counts.keys()]
    rgb_colors = [ordered_colors[i] for i in counts.keys()]

    if (show_chart):
        plt.figure(figsize = (8, 6))
        plt.pie(counts.values(), labels = hex_colors, colors = hex_colors)

    return rgb_colors


# Use this function - input image file, no. of colors to extract, plot pie chart
rgb_colors = get_colors(image, 5, True)